In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

## Summary of Results:


## Problem Setup:

This example is taken from https://arxiv.org/abs/2107.00681 by Hines, Dukes, Diaz-Ordaz, and Vansteelandt (2021) and the empirical evaluation follows https://onlinelibrary.wiley.com/doi/full/10.1002/sim.7628 by Miguel Angel Luque-Fernandez, Michael Schomaker, Bernard Rachet, Mireille E. Schnitzer (2018).

$\psi(P_0) = \mathbb{E}[\mathbb{E}[Y|X=1, Z]] - \mathbb{E}[\mathbb{E}[Y|X=0, Z]]$  is our target estimand - it is the Average Treatment Effect, i.e. the difference in the average outcome $Y$ under treatment $X=1$ and the average outcome $Y$ under no treatment. $X=0$. This can be broken down into two potential outcome goals. So let's simplify it by considering the outcome under treatment:

i.e. $$\psi(P_0) = \int y f(y|1,z)f(z) dy dz$$

Which is the statistical estimand of the mean of $Y^1$ (the quantity is identifiable).

If we assume the distribution from which the densities $f$ derive has been perturbed by a point mass at $(\tilde x, \tilde y, \tilde z)$ then:

$$\psi(P_0) = \int y f_t(y|1,z)f_t(z) dy dz$$

$$ = \int y \frac{f_t(y,1,z)f_t(z)}{f_t(1,z)} dy dz$$

Under the 'parametric submodel' $$P_t = t\delta_x(\tilde x) + (1-t)P_o$$ and where $\delta_x(\tilde x)$ denotes the Dirac delta function s.t. it gives the density of a point mass at $\tilde x$, is zero everywhere else, and integrates to 1.

For our densities we therefore have that:

$$f_t(x,y,z) =  t \delta_{x,y,z}(\tilde x, \tilde y, \tilde z) +  (1-t)f(x,y,z)$$

and, equivalently for $f_t(z)$ and $f_t(1,z)$.

Following the derivation in the paper, the efficient influence function is given as:

$$ \phi_1 = \left . \frac{d\psi (P_t)}{dt} \right \vert_{t=0} = \frac{\mathbb{1}_1(\tilde x)}{\pi(\tilde z,  P)}\{\tilde y - \mathbb{E}(Y|X=1, Z=z)\} + \mathbb{E}(Y|X=1, Z=z) - \psi(P_0)$$

This can be combined with the outcome under no treatment $x=0$ to derive the influence curve for the average treatment effect:

$$\phi_{ATE}=  \phi_1 - \phi_0 - \psi_{ATE}$$

In the expression above, there are a number of 'ground truth' quantities which we need to estimate. Using plug-in estimators we have:


$$ \phi_1(\hat P_n) = \frac{\mathbb{1}_1(\tilde x)}{\pi(\tilde z,  \hat P)}\{\tilde y - m_1(Z, \hat P_n) \} + m_1(Z, \hat P_n) - \psi(\hat P_n)$$

i.e. we have estimators for the propensity score $\pi$, and estimators for the expected outcome $m_1 \approx \mathbb{E}[Y|X=1, Z=z]$

Finally, we want to update out estimate. This can be done a number of ways. E.g.:

$$ \psi(P_0) \approx \psi(\hat P_n) + \frac{1}{n}\sum_{i=1}^n\phi_1(O_i, \hat P_n)$$

Alternatively, we can follow the 'targeted learning' approach and tune the initial estimator for the outcome:

$$m1^* = m1 + \hat \epsilon \left ( \frac{1}{\pi(Z, \hat P_n)} \right ) $$

which solves the influence curve to be equal to zero (i.e. the efficient influence curve):

$$ 0 = \frac{1}{n}\sum_i^n \frac{\mathbb{1}_1(X_i)}{\pi(Z_i, \hat P_n)}\left \{ Y_i - m_1 - \hat \epsilon \frac{1}{\pi (Z_i, \hat P_n)}\right \}$$

This latter approach is the one used in this example.

In [2]:

def sigm(x):
    return 1/(1 + np.exp(-x))

def inv_sigm(x):
    return np.log(x/(1-x))

def generate_data(N, seed):
    np.random.seed(seed=seed)
    z1 = np.random.binomial(1, 0.5, (N,1))
    z2 = np.random.binomial(1, 0.65, (N,1))
    z3 = np.round(np.random.uniform(0, 4, (N,1)),3)
    z4 = np.round(np.random.uniform(0, 5, (N,1)),3)
    X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    Y0 = np.random.binomial(1, sigm(-1 + 0 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    Y = Y1 * X + Y0 * (1-X)
    Z = np.concatenate([z1,z2,z3,z4],1)
    return Z, X, Y, Y1, Y0

## 1. Misspecified Outcome and Treatment Models

This first round demonstrates that targeted learning DOESNT work when both outcome and treatment models are misspecified. Specifically, the DGP contains interactions z2*z4 which we do not pre-specify in the logistic equation models.

Note that each simulation will use the same random seeds.

In [3]:
# First establish ground truth treatment effect:
N = 5000000
Z, x, y, Y1, Y0 = generate_data(N, seed=0)
true_psi = (Y1-Y0).mean()

In [4]:
# Set some params
num_runs = 1000
N = 10000

In [5]:
# Now start simulations
estimates_naive = []
estimates_upd = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'
    
    QZ = Z # misspecify the outcome model by not including interaction term
    GZ = Z[:, :-1]  # misspecify the treatment model in the same way
    
    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)

    biased_psi = (Q1-Q0).mean()

    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = x/(G10)
    H0 = (1-x) / (1 - G10)

    eps0, eps1 = sm.GLM(y, np.concatenate([H0, H1], 1), offset=inv_sigm(Q10[:,0]),
                        family=sm.families.Binomial()).fit().params

    Q0_star = sigm(inv_sigm(Q0) + eps0 * H0)
    Q1_star = sigm(inv_sigm(Q1) + eps1 * H1)
    
    upd_psi = (Q1_star - Q0_star).mean()
    estimates_naive.append(biased_psi)
    estimates_upd.append(upd_psi)

In [6]:
print('Results for misspecified outcome and treatment models....')

estimates_naive = np.asarray(estimates_naive)
estimates_upd = np.asarray(estimates_upd)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive.mean(), ' relative bias:',
      (estimates_naive.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd.mean(), ' relative bias:',
      (estimates_upd.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd.mean() - true_psi)/true_psi * 100, '%')

Results for misspecified outcome and treatment models....
True psi:  0.1956508
naive psi:  0.19777339178968617  relative bias: 1.0848878663854973 %
updated TMLE psi:  0.19802285261944388  relative bias: 1.2123909636167451 %
Reduction in bias: -0.12750309723124786 %


As can be seen in the boxplot above, no bias reduction was achieved. In fact, the update made things worse. This is because the models for *both* the outcome and treatment were misspecified. 

## 2. Misspecified Outcome Model, Correct Treatment Model

Let's create some features accounting for the interaction terms z2*z4 and feed these into the model for the propensity score, thus resulting in misspecification of the outcome model only.

In [7]:
estimates_naive2 = []
estimates_upd2 = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    Z_int = Z[:, 1:2] * Z[:,3:4]
    Z = np.concatenate([Z, Z_int], 1)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'
    
    # Reminder of DGP:
    # X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    # Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    
    QZ = Z[:,:-1]  # misspecify the outcome model by not including interaction term
    GZ = Z[:, 1:]  # correctly specify the treatment model

    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)

    biased_psi = (Q1-Q0).mean()

    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = x/(G10)
    H0 = (1-x) / (1 - G10)

    eps0, eps1 = sm.GLM(y, np.concatenate([H0, H1], 1), offset=inv_sigm(Q10[:,0]),
                        family=sm.families.Binomial()).fit().params

    Q0_star = sigm(inv_sigm(Q0) + eps0 * H0)
    Q1_star = sigm(inv_sigm(Q1) + eps1 * H1)

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive2.append(biased_psi)
    estimates_upd2.append(upd_psi)


In [8]:
print('Results for misspecified outcome model and correctly specified treatment model....')

estimates_naive2 = np.asarray(estimates_naive2)
estimates_upd2 = np.asarray(estimates_upd2)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive2.mean(), ' relative bias:',
      (estimates_naive2.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd2.mean(), ' relative bias:',
      (estimates_upd2.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive2.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd2.mean() - true_psi)/true_psi * 100, '%')


Results for misspecified outcome model and correctly specified treatment model....
True psi:  0.1956508
naive psi:  0.19777339178968617  relative bias: 1.0848878663854973 %
updated TMLE psi:  0.19713150698900234  relative bias: 0.7568111088747517 %
Reduction in bias: 0.3280767575107456 %


The results above show that even if one of the models is misspecified (in the previous case it was the outcome model), we still get a reduction in bias - this is the double robustness property.  

## 3. Outcome and Treatment Models Correctly Specified

Now, let's try again but correctly specify *both* models, whilst comparing against the correctly specified outcome model without the update.

In [9]:
estimates_naive3 = []
estimates_upd3 = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    Z_int = Z[:, 1:2] * Z[:,3:4]
    Z = np.concatenate([Z, Z_int], 1)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'
    
    # Reminder of DGP:
    # X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    # Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
           
    QZ = Z # outcome model correctly specified
    GZ = Z[:, 1:]  # propensity model correctly specified

    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    # note the new reference point is the correctly specified outcome model on its own:
    biased_psi = (Q1-Q0).mean() 
    
    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = x/(G10)
    H0 = (1-x) / (1 - G10)

    eps0, eps1 = sm.GLM(y, np.concatenate([H0, H1], 1), offset=inv_sigm(Q10[:,0]),
                        family=sm.families.Binomial()).fit().params

    Q0_star = sigm(inv_sigm(Q0) + eps0 * H0)
    Q1_star = sigm(inv_sigm(Q1) + eps1 * H1)

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive3.append(biased_psi)
    estimates_upd3.append(upd_psi)
    


In [10]:
print('Results for correctly specified outcome and treatment models....')
print('Note that the naive model is now actually the correct specified outcome model.')
estimates_naive3 = np.asarray(estimates_naive3)
estimates_upd3 = np.asarray(estimates_upd3)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive3.mean(), ' relative bias:',
      (estimates_naive3.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd3.mean(), ' relative bias:',
      (estimates_upd3.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive3.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd3.mean() - true_psi)/true_psi * 100, '%')


Results for correctly specified outcome and treatment models....
Note that the naive model is now actually the correct specified outcome model.
True psi:  0.1956508
naive psi:  0.19537547440087769  relative bias: -0.14072296107264984 %
updated TMLE psi:  0.19555759236038875  relative bias: -0.04763979478298308 %
Reduction in bias: 0.09308316628966676 %


As can be seen, the reduction in bias is not much because we were using the correct outcome model already.

## 4. Outcome Model Correctly Specified, Treatment Model Misspecified 
Let's see what happens when the outcome model is correctly specified, but the treatment model is not, still comparing against the naive model

In [11]:

estimates_naive4 = []
estimates_upd4 = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    Z_int = Z[:, 1:2] * Z[:,3:4]
    Z = np.concatenate([Z, Z_int], 1)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'
    
    # Reminder of DGP:
    # X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    # Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    
    QZ = Z # outcome correctly specified
    GZ = Z[:, :-1]  # propensity model misspecified

    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)

    biased_psi = (Q1-Q0).mean()

    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = x/(G10)
    H0 = (1-x) / (1 - G10)

    eps0, eps1 = sm.GLM(y, np.concatenate([H0, H1], 1), offset=inv_sigm(Q10[:,0]),
                        family=sm.families.Binomial()).fit().params

    Q0_star = sigm(inv_sigm(Q0) + eps0 * H0)
    Q1_star = sigm(inv_sigm(Q1) + eps1 * H1)

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive4.append(biased_psi)
    estimates_upd4.append(upd_psi)


In [12]:
print('Results for correctly specified outcome model and misspecified treatment model....')
print('Note that the naive model is now actually the correct specified outcome model.')
estimates_naive4 = np.asarray(estimates_naive4)
estimates_upd4 = np.asarray(estimates_upd4)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive4.mean(), ' relative bias:',
      (estimates_naive4.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd4.mean(), ' relative bias:',
      (estimates_upd4.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive4.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd4.mean() - true_psi)/true_psi * 100, '%')


Results for correctly specified outcome model and misspecified treatment model....
Note that the naive model is now actually the correct specified outcome model.
True psi:  0.1956508
naive psi:  0.19537547440087769  relative bias: -0.14072296107264984 %
updated TMLE psi:  0.1955532743796033  relative bias: -0.04984677823792245 %
Reduction in bias: 0.0908761828347274 %


Once again, not much bias reduction is achieved because we already started with the correct outcome model.

Note that in practice, we need machine learning algorithms or ensembles (e.g. see super learning https://pubmed.ncbi.nlm.nih.gov/17910531/) which specify a family of models such that the true model is likely to be within that family. Otherwise, we will obviously be unable to specify the correct model without knowing it *a priori*.


## 5. Correct Specification, One-Step update Process
Now, we re-run the last examples, but using the one-step update process to de-bias our initial estimate, instead of estimating estimating epsilon etc. This is therefore different from the typical targeted learning procedure. Let's start with the situation where both outcome and treatment models are correctly specified.

In [13]:
estimates_naive5 = []
estimates_upd5 = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    Z_int = Z[:, 1:2] * Z[:,3:4]
    Z = np.concatenate([Z, Z_int], 1)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'

    # Reminder of DGP:
    # X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    # Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
       
    QZ = Z # outcome correctly specified
    GZ = Z[:, 1:]  # propensity model correctly specified

    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    
    biased_psi = (Q1-Q0).mean()

    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = 1/(G10)
    H0 = 1 / (1 - G10)
    
    D1 = x * H1 * (y - Q1) + Q1 - Q1.mean()
    D0 = (1 - x) * H0 * (y - Q0) + Q0 - Q0.mean()

    Q1_star = Q1 + D1
    Q0_star = Q0 + D0

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive5.append(biased_psi)
    estimates_upd5.append(upd_psi)


In [14]:
print('Results for correctly specified outcome model and treatment models using the one-step update process....')
print('Note that the naive model is now actually the correct specified outcome model.')
estimates_naive5 = np.asarray(estimates_naive5)
estimates_upd5 = np.asarray(estimates_upd5)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive5.mean(), ' relative bias:',
      (estimates_naive5.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd5.mean(), ' relative bias:',
      (estimates_upd5.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive5.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd5.mean() - true_psi)/true_psi * 100, '%')


Results for correctly specified outcome model and treatment models using the one-step update process....
Note that the naive model is now actually the correct specified outcome model.
True psi:  0.1956508
naive psi:  0.19537547440087769  relative bias: -0.14072296107264984 %
updated TMLE psi:  0.19580529903385285  relative bias: 0.07896672738002583 %
Reduction in bias: 0.06175623369262401 %


It can be seen that the results are not quite as good as they were in example number 3 above when using the targeted learning update procedure. Nonetheless, some bias reduction is achieved even when the outcome model is correctly specified to begin with.

## 6. Correctly specified treatment model, Misspecified Outcome Model, one-step update

Nearly there... to cover all bases, we run the situation with the correctly specified treatment model but misspecified outcome model, with the one-step update process.

In [15]:
estimates_naive6 = []
estimates_upd6 = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    Z_int = Z[:, 1:2] * Z[:,3:4]
    Z = np.concatenate([Z, Z_int], 1)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'

    # Reminder of DGP:
    # X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    # Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    
    QZ = Z[:, :-1]  # Misspecify the outcome model
    GZ = Z[:, 1:]  # propensity model correctly specified
    
    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)

    biased_psi = (Q1-Q0).mean()
    
    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = 1/(G10)
    H0 = 1 / (1 - G10)
    
    D1 = x * H1 * (y - Q1) + Q1 - Q1.mean()
    D0 = (1 - x) * H0 * (y - Q0) + Q0 - Q0.mean()

    Q1_star = Q1 + D1
    Q0_star = Q0 + D0

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive6.append(biased_psi)
    estimates_upd6.append(upd_psi)

In [16]:
print('Results for correctly specified treatment model and misspecified outcome model using the one-step update process....')

estimates_naive6 = np.asarray(estimates_naive6)
estimates_upd6 = np.asarray(estimates_upd6)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive6.mean(), ' relative bias:',
      (estimates_naive6.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd6.mean(), ' relative bias:',
      (estimates_upd6.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive6.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd6.mean() - true_psi)/true_psi * 100, '%')


Results for correctly specified treatment model and misspecified outcome model using the one-step update process....
True psi:  0.1956508
naive psi:  0.19777339178968617  relative bias: 1.0848878663854973 %
updated TMLE psi:  0.19580091277418638  relative bias: 0.07672484558527831 %
Reduction in bias: 1.008163020800219 %


Interestingly, there was more bias reduction than in the corresponding example number 2 which used the targeted learning update process.

## 7. Correctly specified outcome, misspecified treatment model, one-step update

Finally, we consider the case where the treatment model is misspecified but the outcome model is correctly specified, using the one-step update process.

In [17]:
estimates_naive7 = []
estimates_upd7 = []
seed = 0
for i in range(num_runs):
    seed += 1
    Z, x, y, Y1, Y0 = generate_data(N, seed)
    Z_int = Z[:, 1:2] * Z[:,3:4]
    Z = np.concatenate([Z, Z_int], 1)
    x_int1 = np.ones_like(x)  # this is the 'intervention data'
    x_int0 = np.zeros_like(x)  # this is the 'intervention data'

    # Reminder of DGP:
    # X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    # Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    
    QZ = Z  # Correctly specify the outcome model
    GZ = Z[:, 0:-1]  # propensity model misspecified
    
    Q = LogisticRegression().fit(np.concatenate([x,QZ], 1), y[:,0])
    Q1 = np.clip(Q.predict_proba((np.concatenate([x_int1, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q0 = np.clip(Q.predict_proba((np.concatenate([x_int0, QZ], 1)))[:, 1:], a_min=0, a_max=1)
    Q10 = np.clip(Q.predict_proba((np.concatenate([x, QZ], 1)))[:, 1:], a_min=0, a_max=1)
   
    biased_psi = (Q1-Q0).mean()

    G = LogisticRegression().fit(GZ, x[:,0])
    G10 = np.clip(G.predict_proba(GZ), a_min=0, a_max=1)[:, 1:]

    H1 = 1/(G10)
    H0 = 1 / (1 - G10)
    
    D1 = x * H1 * (y - Q1) + Q1 - Q1.mean()
    D0 = (1 - x) * H0 * (y - Q0) + Q0 - Q0.mean()

    Q1_star = Q1 + D1
    Q0_star = Q0 + D0

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive7.append(biased_psi)
    estimates_upd7.append(upd_psi)
    
    

In [18]:
print('Results for misspecified treatment model and correctly specified outcome model using the one-step update process....')
print('Note that the naive model is now actually the correct specified outcome model.')
estimates_naive7 = np.asarray(estimates_naive7)
estimates_upd7 = np.asarray(estimates_upd7)
print('True psi: ', true_psi)
print('naive psi: ', estimates_naive7.mean(), ' relative bias:',
      (estimates_naive7.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd7.mean(), ' relative bias:',
      (estimates_upd7.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive7.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd7.mean() - true_psi)/true_psi * 100, '%')


Results for misspecified treatment model and correctly specified outcome model using the one-step update process....
Note that the naive model is now actually the correct specified outcome model.
True psi:  0.1956508
naive psi:  0.19537547440087769  relative bias: -0.14072296107264984 %
updated TMLE psi:  0.1957941411294404  relative bias: 0.07326375841059382 %
Reduction in bias: 0.06745920266205602 %


It can be seen that a little bias reduction was achieved, similar in magnitude to that for example number 4 above which used the targeted learning process.